In [ ]:
import pyttsx3 # text to speech
import datetime 
import speech_recognition as sr
import wikipedia
import webbrowser
import os
import random
import pytz # time zone
import smtplib
from pytube import YouTube
import soundfile as sf
import librosa
from huggingsound import SpeechRecognitionModel
import torch
import subprocess
from transformers import pipeline

In [ ]:
engine = pyttsx3.init('sapi5')
voices = engine.getProperty('voices')
print(voices[0].id)
engine.setProperty('voice',voices[0].id)

In [ ]:
def speak(audio):
    engine.say(audio)
    engine.runAndWait()

In [ ]:
def greet():
    hour = int(datetime.datetime.now().hour)
    if 4 <= hour < 12:
        speak("Good Morning!")
    elif 12 <= hour < 17:
        speak('Good afternoon!')
    else:
        speak('Good evening!')
    speak('I am your assistant. How may I help you?')

In [ ]:
# Function for voice recognition
def takeCommand():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening...")
        r.pause_threshold = 1
        audio = r.listen(source)
    try:
        print("Recognizing...")
        query = r.recognize_google(audio, language='en-in')
        print(f"User said: {query}\n")
    except Exception as e:
        print("Could you repeat that?")
        speak("Could you repeat that?")
        return "None"
    return query

In [ ]:
# Function for sending email
def sendEmail(to, content):
    file_path = r'C:\Users\yash1\OneDrive\Desktop\Yash\3RD YEAR\Semester 5\Mini Project\Project\data.txt'
    if os.path.exists(file_path):
        with open(file_path, 'r') as file:
            file_content = file.read().splitlines()
            if len(file_content) % 2 == 0:  # Check if there are pairs of email and password
                email_password_pairs = [file_content[i:i+2] for i in range(0, len(file_content), 2)]
                print("Select an email ID to send the email:") # Display email options to the user
                speak("Select an email ID to send the email:")
                for index, (email, _) in enumerate(email_password_pairs):
                    print(f"{index + 1}. {email}")
                    
                try:
                    speak("Enter the number corresponding to the email: ") # User selects an email
                    option = int(input("Enter the number corresponding to the email: "))
                    if 1 <= option <= len(email_password_pairs):
                        selected_email, selected_password = email_password_pairs[option - 1]
                        server = smtplib.SMTP('smtp.gmail.com', 587)
                        server.ehlo()
                        server.starttls()
                        server.login(selected_email, selected_password)
                        server.sendmail(selected_email, to, content)
                        server.close()
                    else:
                        print("Invalid option. Exiting.")
                except ValueError:
                    print("Invalid input. Please enter a number. Exiting.")
            else:
                print("File content is not in the expected format (pairs of email and password).")
    else:
        print("File does not exist.")

In [ ]:
# Function for YouTube transcript summarization
def summarizeYouTubeVideo(VIDEO_URL):
    ffmpeg_path = r'C:\ffmpeg\bin\ffmpeg.exe'
    
    yt = YouTube(VIDEO_URL)
    yt.streams.filter(only_audio=True, file_extension='mp4').first().download(filename='ytaudio.mp4')
    command = f'{ffmpeg_path} -i ytaudio.mp4 -acodec pcm_f32le -ar 16000 ytaudio.wav'
    subprocess.run(command, shell=True)
    device="cuda" if torch.cuda.is_available() else "cpu"
    model = SpeechRecognitionModel("jonatasgrosman/wav2vec2-large-xlsr-53-english", device=device)
    input_file = r'C:\Users\yash1\OneDrive\Desktop\Yash\3RD YEAR\Semester 5\Mini Project\ytaudio.wav'
    print(librosa.get_samplerate(input_file))
    stream=librosa.stream(
        input_file,
        block_length=30,
        frame_length=16000,
        hop_length=16000
    )
    for i,speech in enumerate(stream):
      sf.write(f'{i}.wav',speech,16000)
      
    # Audio Transcription
    audio_path=[]
    for a in range(i+1):
      audio_path.append(f'C:\\Users\\yash1\\OneDrive\\Desktop\\Yash\\3RD YEAR\\Semester 5\\Mini Project\\{a}.wav')
    audio_path
    transcriptions=model.transcribe(audio_path)
    full_transcript = ' '
    for item in transcriptions:
      full_transcript+=''.join(item['transcription'])
    full_transcript
    
    # Text Summarization
    summarization = pipeline('summarization')
    
    # Text Chunking before Summarization
    num_iters = int(len(full_transcript)/1000)
    summarized_text = []
    for i in range(0, num_iters + 1):
      start = 0
      start = i * 1000
      end = (i + 1) * 1000
      out = summarization(full_transcript[start:end], min_length = 5, max_length=20)
      out = out[0]
      out = out['summary_text']
      summarized_text.append(out)
    print(summarized_text)
    speak(summarized_text)
    speak("The YouTube video has been summarized.")

In [ ]:
if __name__ == "__main__":
    greet()
    while True:
        query = takeCommand().lower()
        # Opening sites
        sites = [['google', 'google.com'], ['youtube', 'youtube.com'], ['instagram', 'instagram.com'],['facebook','facebook.com'],['amazon','amazon.com']]
        for site in sites:
            if f'open {site[0]}'.lower() in query:
                speak(f'Opening {site[0]}...')
                webbrowser.open(site[1])
                
        # Getting brief info
        if 'wikipedia' in query:
            speak("Searching wikipedia.....")
            query = query.replace("wikipedia","")
            try:
                results = wikipedia.summary(query,sentences=1)
                speak("According to wikipedia")
                print(results)
                speak(results)
            except Exception as e:
                print(f"An error occurred: {str(e)}")
        
        # Playing music from computer
        elif 'play music' in query:
            music_dir = 'C:\\Users\\yash1\\Music\\New Folder'
            songs = [song for song in os.listdir(music_dir) if song.endswith('.mp3')]
            random_song = random.choice(songs)
            print(f"Playing {random_song}")
            os.startfile(os.path.join(music_dir, random_song))
            
        # Getting time
        elif 'time' in query:
            local_time = datetime.datetime.now()
            local_timezone = pytz.timezone('Asia/Kolkata')
            local_time = local_timezone.localize(local_time)
            strTime = local_time.strftime('%H:%M %Z')
            print(f"The current time is {strTime}")
            speak(f"The current time is {strTime}")
        
        # Getting date
        elif 'date' in query:
            date = datetime.datetime.now()
            local_timezone = pytz.timezone('Asia/Kolkata')
            date = local_timezone.localize(date)
            strDate = date.strftime('%d/%m/%Y')
            print(f"The today's date is {strDate}")
            speak(f"The today's date is {strDate}")
            
        # Sending email to someone
        elif 'send mail' in query or 'send email' in query:
            try:
                speak("What to say?")
                content = takeCommand()
                print('Enter the recipient email.')
                speak('Enter the recipient email.')
                to = input("Mail to: ")
                sendEmail(to,content)
                print("Email is sent!")
                speak("Email is sent!")
            except Exception as e:
                print("Sorry, couldn't send mail.")
                speak("Sorry, couldn't send mail.")
        
        # Exiting program
        elif 'exit' in query:
            speak("Exiting the program.")
            break
        
        # Youtube transcript summarizer
        elif 'summarize' in query or 'summarise' in query:
            if 'summarize' in query or 'summarise' in query:
                speak('Summarizing')
                speak("Please provide the YouTube video URL.")
                VIDEO_URL = input('URL here = ')  # Use the takeCommand function to get the URL from user speech input
                if 'youtube.com' in VIDEO_URL:  # Check if the URL contains 'youtube.com' to ensure it's a YouTube video URL
                    summarizeYouTubeVideo(VIDEO_URL)
                else:
                    speak("Invalid URL. Please enter youtube video URL.")